## Support Vector Machines

1. What is SVM?
* Commonly used for classification, specifically binary classification as SVM typically classify data into two groups. For multiclass classification, SVM can still be used, but that means you'd need multiple SVMs. 

* SVMs can also be used as a regression method, maintaining all the main features that characterize the algorithm (maximal margin). However, this is not a common approach for regression.

2. Key concepts of SVM?
* Hyperplane: In SVM classification, the objective is to find a hyperplane that could separate the data points to their corresponding classes. You can think of it as a plane that is located in between two groups of data points to distinguish them.

* Support Vectors: points that are closest to the hyperplane. These are considered "support" vectors as they would influence the position and orientation of the hyperplane. These support vectors will also maximize the margin between the hyperplane.


* Kernel: A tool used by SVM to convert the input space into a higher-dimensional space. There are several kernel functions to choose from, for instance, linear, polynomial and radial basis function (RBF).

3. How does it work exactly?

In the end, we are trying to find the hyperplane with maximized margin, while classifying as many correct data inputs as possible. This can be done using support vectors, which can be formulated as an optimization task. Lagrange multipliers are used to solve this constrained optimization task. Essentially, the maximization will depend only on the dot product of pairs of support vectors. More math info can be found [here](https://towardsdatascience.com/understanding-support-vector-machine-part-1-lagrange-multipliers-5c24a52ffc5e)


4. Pros and cons of SVM?
    * Pros: 
    1. It works really well with a clear margin of separation. 
    2. It is effective in high dimensional spaces.
    3. SVM is effective in cases where the number of dimensions is greater than the number of samples.
    4. SVM is relatively memory efficient

    * Cons: 
    1. It doesn't perform well when we have large data set because the required training time is higher.
    2. SVM does not perform very well when the data set has more noise i.e. target classes are overlapping.
    3. In cases where the number of features for each data point exceeds the number of training data samples, the SVM will underperform.
    4. As the support vector classifier works by putting data points, above and below the classifying hyperplane there is no probabilistic explanation for the classification.